## Data loading

In [1]:
from clef.utils.data_loading import load_datasets
from clef.verification.verify import check_dataset_with_model
from clef.utils.data_loading import write_jsonlines_from_dicts

import os

root_path = '../../'
retrieval_folder = 'setup2-k-10'
out_dir = f'./data-out/{retrieval_folder}'

train, dev = load_datasets(preprocess=True,
                           add_author_name=True,
                           add_author_bio=True,
                           root_path= root_path,)

# ensure out_dir directories exist for later
if not os.path.exists(out_dir):
    os.makedirs(out_dir)
    if not os.path.exists(os.path.join(out_dir, 'eval')):
        os.makedirs(os.path.join(out_dir, 'eval'))

loaded 96 training json lines and 32 dev json lines.


In [2]:
# for RQ3 we add real similarity scores from real TREC file

from clef.utils.data_loading import combine_rumors_with_trec_file_judgements, task5_dir
import os, json

# organizers:
submission_path = os.path.join(root_path,
                               task5_dir,
                               'submission_samples',
                               'KGAT_zeroShot_evidence_English_dev.txt')

# own retrieval:
submission_path = os.path.join(root_path,
                               'clef',
                               'RQ1',
                               'data-out',
                               retrieval_folder,
                               'TERRIER-BM25-qe-dev.trec.txt')

# terrier outputs trec files using " " as separator, our own files use "\t" as separator 
rq3_dataset = combine_rumors_with_trec_file_judgements(dev, submission_path, sep=' ')

print(json.dumps(rq3_dataset[0], indent=4))

{
    "id": "AuRED_142",
    "rumor": "Naturalization decree in preparation: Lebanese passports for sale !",
    "label": "REFUTES",
    "timeline": [
        [
            "https://twitter.com/LBpresidency",
            "1556600039211925504",
            "Account: Lebanese Presidency\nAccount Description: The Presidency of the Lebanese Republic\nText: Today the President of the Republic General Michel Aoun signed 9 laws that were previously approved by the House of Representatives Details at the following link:"
        ],
        [
            "https://twitter.com/LBpresidency",
            "1556559119045332992",
            "Account: Lebanese Presidency\nAccount Description: The Presidency of the Lebanese Republic\nText: President Aoun received the Minister of Foreign Affairs and Expatriates Dr Abdullah Bouhabib and the Minister of Social Affairs Hector Hajjar and discussed with them developments related to the file of displaced Syrians in Lebanon"
        ],
        [
            "

### RQ3 NLI

In [3]:
result = check_dataset_with_model(rq3_dataset, 'bart', debug=True)

outfile = f'{out_dir}/zeroshot-ver-rq3-nli.jsonl'
write_jsonlines_from_dicts(outfile, result)

  0%|          | 0/32 [00:00<?, ?it/s]

Naturalization decree in preparation: Lebanese passports for sale !
	-6.956814701978386 Account: Lebanese Presidency Account Description: The Presidency of the Lebanese Republic Text: President Aoun received the new British Ambassador to Lebanon Hamish Cowell and discussed with him Lebanese-British relations and ways to enhance them in all fields
	-14.582756809697544 Account: Lebanese Presidency Account Description: The Presidency of the Lebanese Republic Text: The Information Office of the Presidency of the Republic denies a false news broadcast by the MTV station about Baabda Palace preparing a decree naturalizing 4 000 people and recalls that it had denied yesterday the false information published by the French magazine Liberation about the same fabricated news
	-6.168744994035234 Account: Lebanese Presidency Account Description: The Presidency of the Lebanese Republic Text: President Aoun received the Lebanese Ambassador to Canada Ambassador Fadi Ziadeh who briefed him on the condi

### RQ3 openai

In [4]:
result = check_dataset_with_model(rq3_dataset, 'openai', debug=True)

outfile = f'{out_dir}/zeroshot-ver-rq3-openai.jsonl'
write_jsonlines_from_dicts(outfile, result)

  0%|          | 0/32 [00:00<?, ?it/s]

Naturalization decree in preparation: Lebanese passports for sale !
	0.0 Account: Lebanese Presidency Account Description: The Presidency of the Lebanese Republic Text: President Aoun received the new British Ambassador to Lebanon Hamish Cowell and discussed with him Lebanese-British relations and ways to enhance them in all fields
	-25.40672960909006 Account: Lebanese Presidency Account Description: The Presidency of the Lebanese Republic Text: The Information Office of the Presidency of the Republic denies a false news broadcast by the MTV station about Baabda Palace preparing a decree naturalizing 4 000 people and recalls that it had denied yesterday the false information published by the French magazine Liberation about the same fabricated news
	0.0 Account: Lebanese Presidency Account Description: The Presidency of the Lebanese Republic Text: President Aoun received the Lebanese Ambassador to Canada Ambassador Fadi Ziadeh who briefed him on the conditions of the Lebanese community

## Evaluation

In [5]:
from clef.utils.scoring import eval_run_custom
from clef.utils.data_loading import task5_dir

import pandas as pd
from IPython.core.display import display_html

import datetime
from csv import writer

sample_submission_file = f'{root_path}/{task5_dir}/submission_samples/KGAT_zeroShot_verification_English_dev.json'

rq3_nli_submission_file = f'{out_dir}/zeroshot-ver-rq3-nli.jsonl'
rq3_openai_submission_file = f'{out_dir}/zeroshot-ver-rq3-openai.jsonl'

ground_truth_file = f'{root_path}/{task5_dir}/data/English_dev.json'

PyTerrier 0.10.0 has loaded Terrier 5.8 (built by craigm on 2023-11-01 18:05) and terrier-helper 0.0.8



eval RQ3

In [6]:
time_now  = datetime.datetime.now().strftime('%d_%m_%Y_%H_%M_%S') 
out_file = f'{out_dir}/eval/RQ3-{time_now}.csv'

eval_data = [
    ['baseline',    *eval_run_custom(sample_submission_file, ground_truth_file, out_file)],
    ['RQ3-nli',     *eval_run_custom(rq3_nli_submission_file, ground_truth_file, out_file)],
    ['RQ3-openai',  *eval_run_custom(rq3_openai_submission_file, ground_truth_file, out_file)],
]

eval_df = pd.DataFrame(eval_data)
eval_df.columns = ['method', 'macro-F1', 'strict-macro-F1']

df_r5  = eval_df[['method', 'macro-F1']].sort_values('macro-F1', axis=0, ascending=False)
df_map = eval_df[['method', 'strict-macro-F1']].sort_values('strict-macro-F1', axis=0, ascending=False)

df1_styler = df_r5.style.set_table_attributes("style='display:inline'").set_caption('macro-F1')
df2_styler = df_map.style.set_table_attributes("style='display:inline'").set_caption('strict-macro-F1')

eval_df.to_csv(out_file)

display_html(df1_styler._repr_html_()+df2_styler._repr_html_(), raw=True)


,method,macro-F1
2,RQ3-openai,0.948413
0,baseline,0.508159
1,RQ3-nli,0.182099
,method,strict-macro-F1
2,RQ3-openai,0.932712
0,baseline,0.508159
1,RQ3-nli,0.156736
